# Rabin-Karp

Реализовать поиск подстроки в строке с помощью алгоритма Рабина-Карпа. 

К решению нужно прикрепить отчет с

* объяснением rolling hash
* оценкой сложности итогового алгоритма

Тесты продолжаем писать.

# Import

In [5]:
import os

while os.getcwd().split("/")[-1] != "algorithms_python":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [6]:
import random
import time
from collections import deque
from typing import Any, Callable, Dict, List, Optional

import numpy as np

# Solution

![rabin_karp](../imgs/rabin_karp.png)

Логика:
- Вычисляем хеш паттерна
- Вычисляем хеш для первой подстроки текста той же длины
- Сравниваем хеши:
    - Если хеши разные — подстрока точно не совпадает с паттерном
    - Если хеши совпали — проверить подстроку на полное совпадение
    - Сдвигаем окно на одну позицию и пересчитываем хеш

**Rolling Hash**
Идея в том, что вместо того чтобы каждый раз вычислять хэш с нуля (это стоило бы O(n) для подстроки длины n), мы пересчитываем его за O(1) на основе предыдущего значения.

**Для первой подстроки текста**

> $hash(S) = (S[0] * b^{(n-1)} + S[1] * b^{(n-2)} + ... + S[n-1] * b^0) \% p$
где 
- S[i] - числовое значение i-ого символа,
- b - основание
- p - для предотвращения переполнения хэша

**Для следующей подстроки текста**
> $new\_hash = ((old\_hash - S[i] * b^{(n-1)}) * b + S[i+n]) \% p$

Например, ```S[i] * b^(n-1)``` — вклад самого старшего символа в старом хэше

Например, для "123" самый старший символ "1" имеет вес b^2 = 100

Удаляем самый старый символ -> ```old_hash - S[i] * b^(n-1)```, получается -> ```123 - 1*100 = 23```

Сдвигаем все оставшиеся символы на один разряд влево
```(...) * b``` -> ```23 * 10 = 230```

Добавляем новый символ
```(...) + S[i+m]``` —> ```230 + 4 = 234```

Т.к. мы не пересчитываем хэш на каждом окне, то получается что вычисление хэша подстроки равно O(1), а вычисление хэша для всей строки, в которой мы ищем паттерн равно O(n), где n длина строки. Затем когда найдем совподающие хэши идет перебор всех элементов паттерна и окна размером m -> O(m). В итоге получается Time Complextiy = O(n+m).

https://www.geeksforgeeks.org/dsa/rabin-karp-algorithm-for-pattern-searching/

In [ ]:
def rabin_karp_search(text: str, pattern: str) -> list[int]:
    base = 37
    prime = 1e9 + 7

    n = len(text)
    m = len(pattern)

    if m == 0 or n < m:
        return []

    pattern_hash = 0
    text_hash = 0
    h = 1 

    for i in range(m - 1):
        h = (h * base) % prime

    # хэши для pattern и первого окна text
    for i in range(m):
        pattern_hash = (base * pattern_hash + ord(pattern[i])) % prime
        text_hash = (base * text_hash + ord(text[i])) % prime

    result = []

    # идем по тексту окном размера m
    for i in range(n - m + 1):
        if pattern_hash == text_hash:
            # При совпадении хэшей проверяем посимвольно
            if text[i : i + m] == pattern:
                result.append(i)

        # Вычисляем хэш для следующего окна
        if i < n - m:
            text_hash = (
                base * (text_hash - ord(text[i]) * h) + ord(text[i + m])
            ) % prime

            if text_hash < 0:
                text_hash += prime

    return result

In [10]:
text1 = "ABABDABACDABABCABAB"
pattern1 = "ABABCABAB"
result1 = rabin_karp_search(text1, pattern1)
print(f"pattern: '{pattern1}'\ntext: '{text1}'")
print(result1)
print()

text2 = "AABAACAADAABAABA"
pattern2 = "AABA"
result2 = rabin_karp_search(text2, pattern2)
print(f"pattern: '{pattern2}'\ntext: '{text2}'")
print(result2)
print()

text3 = "Hello, world!"
pattern3 = "Python"
result3 = rabin_karp_search(text3, pattern3)
print(f"pattern: '{pattern3}'\ntext: '{text3}'")
print(result3)
print()

text4 = "Some text"
pattern4 = ""
result4 = rabin_karp_search(text4, pattern4)
print(f"pattern: '{pattern4}'\ntext: '{text4}'")
print(result4)
print()

text5 = "ABCABCABCABCABC"
pattern5 = "ABC"
result5 = rabin_karp_search(text5, pattern5)
print(f"pattern: '{pattern5}'\ntext: '{text5}'")
print(result5)

pattern: 'ABABCABAB'
text: 'ABABDABACDABABCABAB'
[10]

pattern: 'AABA'
text: 'AABAACAADAABAABA'
[0, 9, 12]

pattern: 'Python'
text: 'Hello, world!'
[]

pattern: ''
text: 'Some text'
[]

pattern: 'ABC'
text: 'ABCABCABCABCABC'
[0, 3, 6, 9, 12]
